In [1]:
import pandas as pd
import numpy as np

df_titanic = pd.read_csv("titanic.csv")
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [129]:
def describe_df(dataframe):
    dic_columnas = {}
    for columna in dataframe.columns:
        a = dataframe[columna].dtype
        b = round((len(dataframe.loc[dataframe[columna].isna()])/len(dataframe)*100),2)
        c = dataframe[columna].nunique()
        d = round(dataframe[columna].nunique()/len(dataframe)*100, 2)
        dic_columnas[columna] = [a, b, c, d]

    indice = ["DATA_TYPE", "MISSINGS (%)", "UNIQUE_VALUES", "CARDIN (%)"]
    df = pd.DataFrame(dic_columnas, index = indice)
    return df

In [ ]:
describe_df(df_titanic)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
DATA_TYPE,int64,int64,object,float64,int64,int64,float64,object,object,object,bool,object,object,object,bool
MISSINGS (%),0.0,0.0,0.0,19.87,0.0,0.0,0.0,0.22,0.0,0.0,0.0,77.22,0.22,0.0,0.0
UNIQUE_VALUES,2,3,2,88,7,7,248,3,3,3,2,7,3,2,2
CARDIN (%),0.22,0.34,0.22,9.88,0.79,0.79,27.83,0.34,0.34,0.34,0.22,0.79,0.34,0.22,0.22


In [ ]:
def tipifica_variables (df_in, umbral_categoria = 10, umbral_continua=20):
    diccion = {}
    for columna in df_in.columns:
      if df_in[columna].nunique() == 2:
        diccion[columna] = "Binaria"
      if df_in[columna].nunique() > 2 and df_in[columna].nunique() < umbral_categoria:
        diccion[columna] = "Categórica"
      if df_in[columna].nunique() >= umbral_categoria:
        diccion[columna] = "Numérica discreta"
      if (df_in[columna].nunique()/len(df_in[columna]) * 100) >= umbral_continua:
          diccion[columna] = "Numérica continua"
        
    categoria = pd.Series(diccion)
    df = pd.DataFrame(categoria)
    
    return df.rename(columns={0:"Categoria"})

In [ ]:
tipifica_variables(df_titanic)


,Categoria
survived,Binaria
pclass,Categórica
sex,Binaria
age,Numérica discreta
sibsp,Categórica
parch,Categórica
fare,Numérica continua
embarked,Categórica
class,Categórica
who,Categórica


In [ ]:
from scipy.stats import pearsonr
import matplotlib as plt
import seaborn as sns
def get_features_num_regression(dataframe, target_col,umbral_corr, pvalue = None):
    lista_columnas = []
    for col in dataframe.columns:
        if dataframe[col].dtype != 'object':
            corrs = pearsonr(dataframe[target_col], dataframe[col])
            if pvalue != None:
                if np.abs(corrs[0]) > umbral_corr and corrs[1] <= pvalue:
                    lista_columnas.append(col)
            else:
                if np.abs(corrs[0]) > umbral_corr:
                    lista_columnas.append(col)
    
    lista_columnas.insert(0, target_col)
    corr_matrix = lista_columnas.corr()
    figura = plt.figure(figsize= (8,8))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, linewidths=.5) # el cmap es el rango de colores usado para representar "el calor"
    plt.title('Matriz de Correlación')
    plt.xticks(rotation=45)  # Rota las etiquetas de las x si es necesario
    plt.yticks(rotation=45)  # Rota las etiquetas de las y si es necesario


    
    return lista_columnas, figura
            